# Solucao Extracao pdf v2

0_solucao_extracao_pdf_v2.ipynb


## 1. Modulos, config e dicionarios

### 1. Modulos 

In [ ]:
import os
import csv

import json
import shutil
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path

import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract

import fitz  # Módulo PyMuPDF
import re
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd

from PIL import ImageFont
from PIL import Image, ImageDraw



import zipfile

from outlook_msg import Message
import extract_msg

import locale
from datetime import datetime, timezone, timedelta


### 2. Config

In [ ]:

# 1. path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 2. path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "pipeline_extracao_documentos/3_tratamento_excecoes/pdf_aguardando_processar"





# path para e-mails recebidos com documentos
email_recebido_documento_path = "pipeline_extracao_documentos/1_emails"





# 3. path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"
# 4. path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"






# 5. path para imagem padrao
image_resized_path = 'pipeline_extracao_documentos/6_geral_administacao/images/processadas'

# 6. path para log
log_path = 'pipeline_extracao_documentos/6_geral_administacao/logs'


# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 8. path para NFs processadas
nf_processada_path = "pipeline_extracao_documentos/5_documentos_processados"



#### paths de objetos para criacao/gestao (dicionarios/datasets)

# 9. path para modelos
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v5_mage1.xlsx"

# 10. path para dicionario de modelos
model_dict_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/models.csv"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"







# VERIFICAR
tgt_imagens = "pipeline_extracao_documentos/images"

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

In [ ]:
# 1. Caminho do arquivo uma mensagem especifica
msg_dir_path = 'pipeline_extracao_documentos/1_emails'

# 2. Path para arquivos atachados compactados
msg_attachment_zip = 'pipeline_extracao_documentos/1_emails/attachments'

# 3. Path para documentos atachados:
documentos_extracao_path = 'pipeline_extracao_documentos/2_documentos_para_extracao'

batch_name = "Batch_1"

df_processamento = pd.read_csv('pipeline_extracao_documentos/6_geral_administacao/datasets/df_processamento.csv')

df_processamento.head()

## 1. Consistencias

### 1. Tratamento dos e-mails

In [ ]:
for root, dirs, files in os.walk(msg_dir_path):
    #print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.msg'):
            email_message = file
            email_path = os.path.join(root, file)
            print(f'email: {email_message}, email_path: {email_path}')
            
            
msg = extract_msg.Message(email_path)

msg_raw_sender = msg.sender

parts = msg_raw_sender.rsplit('<', 1)

msg_email_address = parts[1].strip('<>')

msg_sender = parts[0].strip(' ')

msg_subject = msg.subject


msg_body = msg.body

# Defina a localização para pt_BR
locale.setlocale(locale.LC_TIME, "pt_BR.utf8")


# String original
original_date_str = msg.date



    
    

In [ ]:
original_date_str

In [ ]:
# Utiliza expressão regular para extrair os componentes
match = re.search(r'(\d{2}) (\w{3}) (\d{4}) (\d{2}):(\d{2}):(\d{2}) ([+-]\d{4})', original_date_str)
if match:
    day, month_str, year, hour, minute, second, tz_offset = match.groups()
    
    month_map_en = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
    month_map_pt = {'jan': 1, 'fev': 2, 'mar': 3, 'abr': 4, 'mai': 5, 'jun': 6, 'jul': 7, 'ago': 8, 'set': 9, 'out': 10, 'nov': 11, 'dez': 12}

    # Detectar idioma do campo de data (um exemplo simples, você pode fazer algo mais robusto)
    if 'Jan' in original_date_str or 'Feb' in original_date_str or 'Mar' in original_date_str:
        month_map = month_map_en
    else:
        month_map = month_map_pt

# Continua com o resto do seu código

    month = month_map.get(month_str, 0)
    
    # Cria um objeto datetime
    dt = datetime(year=int(year), month=month, day=int(day),
                  hour=int(hour), minute=int(minute), second=int(second))
    
    # Ajusta o fuso horário
    offset_minutes = int(tz_offset) // 100 * 60 + int(tz_offset) % 100
    dt = dt.replace(tzinfo=timezone(timedelta(minutes=offset_minutes)))


    # Formata a data para o formato desejado (pt_BR)
    formatted_date_str = dt.strftime("%d/%m/%Y %H:%M:%S")
    #print(f"Data e horário formatados: {formatted_date_str}")

    # Formata apenas a data
    formatted_date = dt.strftime("%d/%m/%Y")
    #print(f"Data: {formatted_date}")

    # Formata apenas o horário
    formatted_time = dt.strftime("%H:%M:%S")
    #print(f"Horário: {formatted_time}")
    
    print(f'\nmsg_sender: {msg_sender}, msg_email_address: {msg_email_address}, msg_subject: {msg_subject} data: {formatted_date}, horario: {formatted_time} data completa: {formatted_date_str}')
else:
    print("Não foi possível fazer o parse da data.")

<h3 Salvando os attachments do e-mail </h3>

<h3> Tratando os anexos, zips </h3>

In [ ]:
import os
from outlook_msg import Message

# Caminho da pasta onde você quer salvar os anexos
pasta_destino = msg_attachment_zip

# Verifica se a pasta existe; se não, cria ela
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

with open(email_path) as msg_file:
    msg = Message(msg_file)

# Contents are the plaintext body of the email
#contents = msg.body


total_attch = len(msg.attachments)

print(total_attch)

arquivos_zip = []
i = 0
# Loop para salvar cada anexo
for i in range(total_attch):
    attachment = msg.attachments[i]
    caminho_completo_anexo = os.path.join(pasta_destino, attachment.filename)
    arquivos_zip.append(attachment.filename)
    print(caminho_completo_anexo)
    with attachment.open() as attachment_fp, open(caminho_completo_anexo, 'wb') as output_fp:
        output_fp.write(attachment_fp.read())



<mark> Extraindo documentos do ZIP </mark>

In [ ]:
# Diretório onde você quer salvar os arquivos extraídos
output_dir = os.path.join(documentos_extracao_path, batch_name)

folder_file_dict = {}

for root, dirs, files in os.walk(msg_attachment_zip):
    #print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.zip'):
            zip_file = file
            zip_file_path = os.path.join(root, file)
            print(zip_file_path)
            
            # Obtém o nome base do arquivo ZIP para usar como subdiretório
            zip_basename = os.path.splitext(os.path.basename(zip_file_path))[0]
            
            print(zip_basename)
            # Obtém o nome base do arquivo ZIP para usar como subdiretório
            zip_basename = os.path.splitext(os.path.basename(zip_file_path))[0]

            # Cria o subdiretório com base no nome do arquivo ZIP
            root_output_dir = os.path.join(output_dir, zip_basename)
            if not os.path.exists(root_output_dir):
                os.makedirs(root_output_dir) # estou criando o diretorio caso nao exista

            # Dicionário para guardar o nome da pasta e os arquivos associados
            

            # Abre o arquivo ZIP
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                for member in zip_ref.namelist():
                    # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
                    parts = member.rsplit('\\', 1)
                    folder_name = parts[0] if len(parts) > 1 else ''
                    filename = parts[-1]

                    if filename:  # ignora diretórios
                        # Adiciona ao dicionário
                        folder_file_dict.setdefault(folder_name, []).append(filename)

                        # Cria um subdiretório se ele não existir
                        sub_dir = os.path.join(root_output_dir, folder_name)
                        if not os.path.exists(sub_dir):
                            os.makedirs(sub_dir)

                        # Salva o arquivo no subdiretório especificado
                        source = zip_ref.open(member)
                        target_path = os.path.join(sub_dir, filename)
                        with open(target_path, "wb") as target:
                            target.write(source.read())


folder_file_dict

### 2. Dicionarios e Dataframe

In [ ]:
folder_names = []
file_counts = []
file_names = []

# Iterar sobre o dicionário para coletar informações
for folder, files in folder_file_dict.items():
    folder_names.append(folder)
    file_counts.append(len(files))
    file_names.append(files)


# Suponha que folder_file_dict é algo como {'pasta1': 'arquivo1.pdf', 'pasta2': 'arquivo2.pdf'}
folder_names = list(folder_file_dict.keys())
file_names = list(folder_file_dict.values())

In [ ]:
dfs = {} 
# Criar o DataFrame do batch
dfs[batch_name] = pd.DataFrame({
    "Dt_hora": [formatted_date_str],
    "Assunto": [msg_subject],
    "Arquivos_zip": [arquivos_zip],
    "Pasta": folder_names,
    "Quantidade de Documentos": [file_counts],
    "documentos": file_names,
    "De": [msg_sender],
    "batch": [batch_name],
    "email": [msg_email_address],
    "Data": [formatted_date],
    "Hora": [formatted_time],
})

df_batch = dfs[batch_name]

In [ ]:
df_processamento = dfs[batch_name]

In [ ]:
# Anexando o novo DataFrame ao original
df_processamento = df_processamento.append(df_batch, ignore_index=True)

df_processamento.head()

In [ ]:
# Anexando o novo DataFrame ao original
df_processamento = df_processamento.append(df_batch, ignore_index=True)

df_processamento.head()




In [ ]:
# Salvando o DF para csv
df_processamento.to_csv('pipeline_extracao_documentos/datasets/df_processamento.csv')

In [ ]:
# gerar excel a partir do df

df_main.to_excel("nome_do_arquivo.xlsx", index=False)


### 3. Analise dos PDFs

In [ ]:
def convertResizeAnalise_1page(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    resized_pages = []
    for page in pages:
        resized_page = page.resize((2067, 2923))
        resized_pages.append(resized_page)
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

In [ ]:
# 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable_analise(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        pages = pdf_document.page_count
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf, pages
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

In [ ]:
# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

In [ ]:
# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

In [ ]:
# 5. Interacao para pesquisar prefeitura
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

In [ ]:
# Funcao importante - process_line

def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None

In [ ]:
# 1. dicionario para ocorrencias dos documentos pesquisados
analise_doc_nf = {}
file_data = []
# 2. caminho usado dinamicamente para funcao poder ser utizada em outras area da estrutura de diretorios
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
# Inicialização do DataFrame
colunas = ["documento", "pdf", "batch", "pesquisavel", "paginas", "metadata"]
analise_df = pd.DataFrame(columns=colunas)
i = 0
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    
    for file in files:
        file_path = os.path.join(root, file)
        dados_pdf = None  # Inicialização fora do try/catch
        
        if file.lower().endswith('.pdf'):
            documento_pdf = True

            pesquisavel, metadados, paginas = is_pdf_searchable_analise(file_path)
                
            if not pesquisavel:
                
                if paginas == 1:
                    #1. Converto para imagem
                    image_2work, name_image_2work = convertResizeAnalise_1page(file, file_path, image_resized_path)


                    #2_frame_dados_prestador:  0	552	2067	785


                    #2. Defino cordenandas, string_pesquisa e label  550	380	1357	555
                    v_x0 = 0#406
                    v_y0 = 552#0
                    v_x1 = 2067#1540
                    v_y1 = 785#380
                    string_pesquisa = "Nome/Razão Social:"
                    label = "testando pesquisas regex"
                    
                    #3. Print do teste
                    print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas} | string_pesquisa: "{string_pesquisa}"\ncoordenadas ocr: v_x0 = {v_x0}, v_y0 = {v_y0}, v_x1 = {v_x1}, v_y1 = {v_y1}\n')
                    
                    #4. Processo OCR
                    text_splited, text_frame = ocr_RasterPDF_free(image_2work, v_x0, v_y0, v_x1, v_y1)
                                        
                    #5. Itero sobre text_splited (lista)
                    a = 0
                    for texto_spl in text_splited:
                        print(f'texto_spl linha {a}: {texto_spl}')
                        #6. chamo funcao process_line
                        verificacao_texto = process_line(texto_spl, string_pesquisa, label)
                        if verificacao_texto:
                            print(f'\n1. nro: {i} | file: {file}\nverificacao_texto: {verificacao_texto}\n')
                        a +=1    
                    
                   
                    #6. Imprimo valores de text_splited
                    print(f'2. text_splited:\n{text_splited}\n')
                    print(f'\n\n=============================================================================================')
                    
                    
                    i +=1
                    
                    
                
            

In [ ]:
# 1. dicionario para ocorrencias dos documentos pesquisados
analise_doc_nf = {}
file_data = []
# 2. caminho usado dinamicamente para funcao poder ser utizada em outras area da estrutura de diretorios
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)


# Inicialização do DataFrame
colunas = ["documento", "pdf", "batch", "pesquisavel", "paginas", "metadata"]
analise_df = pd.DataFrame(columns=colunas)

for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    
    for file in files:
        file_path = os.path.join(root, file)
        dados_pdf = None  # Inicialização fora do try/catch
        
        if file.lower().endswith('.pdf'):
            documento_pdf = True
            try:
                pesquisavel, metadados, paginas = is_pdf_searchable_analise(file_path)
                
                
                                
                if not pesquisavel:
                    if paginas == 1:
                        image_2work, name_image_2work = convertResizeAnalise_1page(file, file_path, image_resized_path)
                        text_splited, text_frame  = ocr_RasterPDF(name_image_2work)
                    
                    
                    print(f'\ntext_splited: {text_splited}, text_frame: {text_frame}')
                    
                    #print(file, "nao e pesquisavel")
                
                
            except Exception as e:
                print(f"Erro ao verificar o PDF: {e}")
            # Adicionando uma nova linha ao DataFrame
            nova_linha_df = pd.DataFrame([{
                "pdf": documento_pdf,
                "documento": file,
                "batch": batch_name,
                "pesquisavel": pesquisavel,
                "paginas": paginas,
                "metadata": metadados
            }])
            
            analise_df['pdf'] = analise_df['pdf'].astype(bool)
            analise_df['pesquisavel'] = analise_df['pesquisavel'].astype(bool)

            analise_df = pd.concat([analise_df, nova_linha_df], ignore_index=True)

        else:
            documento_pdf = False
            # Adicione lógica para outros tipos de arquivo, se necessário

In [ ]:
analise_df

In [ ]:
# 1. Funcao de conversao e resize do documento
def convertResize_analise(nome_documento, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(nome_documento)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

In [ ]:
def pequisaModel(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
            return  nome_prefeitura
        else:
            raise ValueError("Nao acho nome de prefeitura")

<mark> Tratamento de exceçoes </mark>

In [ ]:
def minha_funcao():
    meu_dict = {"erros": []}
    
    try:
        valor = "teste"
        if valor == "teste":
            try:
                numero = "outro dado"
                resultado = int(numero)
            except ValueError as e:
                meu_dict["erros"].append(f"Erro na primeira parte: {str(e)}")
                    
        valor = int("texto")  # Isso vai lançar ValueError
    except ValueError as e:
        meu_dict["erros"].append(f"Erro na segunda parte: {str(e)}")
    
    if meu_dict["erros"]:
        print("Erros encontrados.")
    
    return meu_dict


In [ ]:

resultado = minha_funcao()
print("Resultado: ", resultado)

In [ ]:
# move NF processadas ok
def move_pdf_processed_ok(document_path, nf_processada_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(nf_processada_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message

In [ ]:
page_to_checke.resize(2067, 2923)

In [ ]:
pdf_document.close()

In [ ]:
pages[1]

In [ ]:
pdf_document = fitz.open(documento_path)

### Logging

In [ ]:
import logging
import datetime

# Get the current date for filename
current_date = datetime.datetime.now().strftime("%Y-%m-%d")


log_filename = f"log_{current_date}.txt"

log_file_path = os.path.join(log_path, log_filename)


# Configure the logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%d/%m/%Y %H:%M:%S',  
    filename=log_file_path
)

### 3. Dicionarios de modelos

In [ ]:
# 2. Leitura do arquivo CSV e criação do dicionário modelos
def create_model_dictionary(model_dict_path):
    model_dictionary = {}
    with open(model_dict_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            prefeitura_name = row['prefeitura']
            model_name = row['model']

            if prefeitura_name not in model_dictionary:
                model_dictionary[prefeitura_name] = model_name
            
            #model_dictionary[prefeitura_name].append(model_name)
    
    return model_dictionary


# 3. Cria o dict de modelos
model_dict = create_model_dictionary(model_dict_path) 





#Le a planilha e cria do DF
frames_nf_v3_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'boundaries']
sections_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'section']
frames_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'frame']
sframe_fields_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v3_df[frames_nf_v3_df['type'] == 'field_box']

In [ ]:
frames_info

### 4. CNAE Dict

In [ ]:
mage_cnae_x_item_servico_df = pd.read_excel('pipeline_extracao_documentos/datasets/MAGE_CNAE_X_ITEM_SERVICO_V1.xlsx')

# Creating a dictionary for CNAE codes and descriptions
cnae_dict = dict(zip(mage_cnae_x_item_servico_df['cnae'], mage_cnae_x_item_servico_df['descricao_cnae']))
item_servico_dict = dict(zip(mage_cnae_x_item_servico_df['item_servico'], mage_cnae_x_item_servico_df['descricao_item_servico']))

In [ ]:
item_servico_dict[7.02].upper()
cnae_dict[4313400].upper()

In [ ]:
cnae_dict[4313400].upper()

In [ ]:
cnae_dict[3812200]

In [ ]:
mage_cnae_x_item_servico_df.head()

In [ ]:
from collections import defaultdict

# Initialize a default dictionary with list as the default factory
cnae_to_item_servico_dict = defaultdict(list)

# Iterate through the DataFrame and populate the dictionary
for index, row in mage_cnae_x_item_servico_df.iterrows():
    cnae = row['cnae']
    cnae_cod = row['cnae_cod']
    descricao_cnae = row['descricao_cnae']
    item_servico = row['item_servico']
    descricao_item_servico = row['descricao_item_servico']
    cnae_to_item_servico_dict[cnae_cod].append(item_servico)




In [ ]:
cnae_to_item_servico_dict

In [ ]:
len(cnae_to_item_servico_dict)

## 2. Funçoes gerais de imagem e consistencias

### 2.1. Funcoes de tratamento de imagem e  PDF

In [ ]:
# 1. Funcao de conversao e resize do documento
def convertResize(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

# 2. Pesquisa prefeitura no documento
def pequisaModel(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura
   
# 3. Ajusta o filename tirando caracteres especiais 
def conv_filename(title):
    
    # Divida o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    # Adicione a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 4. Ajusta o filename tirando caracteres especiais e a
def conv_filename_no_ext(title):
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename     

# 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False


In [ ]:
frames_info

### 2.2 Funcoes de suporte e extracao

In [ ]:
# Funçao de formatacao de numeros
"""def format_number(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)"""
    
    
def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)
    

In [ ]:
################### Funcoes suporte para extracao

def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


def format_number(number_str):
    # Remove unwanted characters
    number_str = number_str.replace('R$', '').replace('%', '')

    # Extract the last numeric part using regex
    number_str = re.findall(r'[\d,\.]+', number_str)[-1]

    # Replace commas with dots (if you use commas as decimal separators)
    number_str = number_str.replace(',', '.')

    # Split by dots and rejoin, treating all but the last dot as thousands separators
    parts = number_str.split('.')
    if len(parts) > 1:
        number_str = ''.join(parts[:-1]) + '.' + parts[-1]

    return float(number_str)





    # Dicionário para armazenar o texto final processado para cada field
    final_extracted_data = {}

    # Itere pelos dados de texto extraídos e processe o texto para manter apenas os valores
    for reference, extracted_text in extracted_text_data_default_lang.items():
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        # Formate o valor usando a função format_number
        value = format_number(value)
        # Armazene o valor processado no dicionário final
        final_extracted_data[reference] = value

    # Display the final extracted data for the first few fields
    list(final_extracted_data.items())[:15]
    
    
def extract_fields_prestador_cnpj(text): # Função para extrair campos e valores dentro de um retângulo
    nf_data_prestador_cnpj = {}
    
    nf_data_prestador_cnpj['secao'] = "2. PRESTADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))


    # Extrair Telefone
    telefone_str = None
    
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador_cnpj['telefone'] = telefone_str
    else:
        nf_data_prestador_cnpj['telefone'] = None   
    
    
    return nf_data_prestador_cnpj 


def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    
    nf_data_tomador_cnpj['secao'] = "3.TOMADOR DE SERVIÇO"
    
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj


def extract_fields_tomador_inscricao(text):
    nf_data_tomador_inscricao = {}
    
    # Extrair RG    
    rg_match = re.search(r'RG:\s+(.+)', text)   
    if rg_match:
        rg_str = rg_match.group(1)
        if rg_str == 'Inscrição Estadual:':
            nf_data_tomador_inscricao['rg'] = None  # Valor padrão quando não há correspondência
        else:    
            nf_data_tomador_inscricao['rg'] = rg_match.group(1)  
 
        
                
    # Extrair Inscrição Estadual
    inscricao_estadual_match = re.search(r'Inscrição Estadual:\s+(.+)', text)
    if inscricao_estadual_match:
        inscricao_estadual_str = inscricao_estadual_match.group(1)
        if inscricao_estadual_str == '':
            nf_data_tomador_inscricao['inscricao_estadual'] = None
        else:    
            nf_data_tomador_inscricao['inscricao_estadual'] = inscricao_estadual_match.group(1)   
    else: 
        nf_data_tomador_inscricao['inscricao_estadual'] = None           

    return nf_data_tomador_inscricao


def extract_fields_tomador_dados(text):
    
    nf_data_tomador = {}

    # Dividir o texto em linhas
    linhas = text.split('\n')

    # Inicializar variáveis para armazenar os valores
    nome_razao_social = None
    endereco = None
    email = None

    # Iterar pelas linhas para identificar os campos e valores
    i = 0
    while i < len(linhas):
        linha = linhas[i]
        if "Nome/Razão Social:" in linha:
            nome_razao_social = linhas[i + 1].strip()
            i += 2
        elif "Endereço:" in linha:
            endereco = linhas[i + 1].strip()
            i += 2
        elif "E-mail:" in linha:
            email = linhas[i + 1].strip()
            i += 2
        else:
            i += 1


    nf_data_tomador['razao_social'] = nome_razao_social
    nf_data_tomador['endereco'] = endereco
    nf_data_tomador['email'] = email
    
    return nf_data_tomador

###################Funcoes Modelo e Frames

# 1 - CABECALHO
def processa_cabecalho():

    nf_data_cabecalho = {}
    model = "mage_1"
    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == model]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "1_frame_prefeitura_nf" or frame_father == "1_frame_dados_nf":
                
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                
                seq = row_frame['seq']
                #print(f'\nLoop 1: {frame_model}, seq.: {seq}, row_frame[label]: {frame_father}\n')
                nf_data_cabecalho['secao'] = "1 - CABECALHO"
                
                for index_field, row_field in sframe_fields_info.iterrows():
                    item_son = row_field['father']
                    frame_father = row_frame['label']
                    if item_son == frame_father:

                        model_value = row_field['model']
                        type_value = row_field['type']
                        label_value = row_field['label']
                        reference_value = row_field['reference']
                        seq_value = row_field['seq']
                        father_value = row_field['father']
                        # ... Select specific fields ...
                        if label_value == "nome_prefeitura" and model_value == frame_model: 
                            #print(f'  Loop 2: {model_value}, seq.: {seq_value}, type_value: {type_value}, label_value: {label_value}')
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                                    
                        elif label_value == "secretaria" and model_value == frame_model: 
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                                    
                        elif label_value == "tipo_nota_fiscal" and model_value == frame_model:
                            values = extracted_text_frame.split('\n')
                            for value in values:
                                result = process_line(value, reference_value, label_value)
                                if result:
                                    nf_data_cabecalho.update(result)
                        
                        #Extraçao de Dados da NF            
                        elif father_value == "1_frame_dados_nf" and model_value == frame_model:
                            values = extracted_text_frame.split('\n')
                            data_list = [item for item in values if item != '']
                            
                            data_dict = {}
                            for i in range(0, len(data_list), 2):
                                key = data_list[i]
                                value = data_list[i+1]
                                data_dict[key] = value

                            # You can now access the values using the corresponding labels
                            nro_nota_fiscal = data_dict['Número da Nota:']
                            nf_data_cabecalho['numero_nota_fiscal'] = data_dict['Número da Nota:']
                            nf_data_cabecalho['competencia'] = data_dict['Competência:']
                            nf_data_cabecalho['dt_hr_emissao'] = data_dict['ata e Hora da Emissão:']
                            nf_data_cabecalho['codigo_verificacao'] = data_dict['Código Verificação:']
                            
    return nf_data_cabecalho, nro_nota_fiscal 

# 2 - PRESTADOR DE SERVIÇO
def processa_prestador():

    nf_data_prestador = {}
    
    warning = {}

    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == modelo]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "2_frame_cnpj_prestador":
                
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                nf_data_prestador = extract_fields_prestador_cnpj(extracted_text_frame)
                
            elif frame_father == "2_frame_inscricao_prestador":
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                # 1. Prrestador de Servico - INSCRICAO ESTADUAL/MUNICIPAL
                nf_data_prestador_incricao = {}

                # Dividir o texto em linhas
                linhas = extracted_text_frame.split('\n')

                # Inicializar variáveis para armazenar os valores
                inscricao_municipal = None
                inscricao_estadual = None


                # Initialize variables
                inscricao_municipal = ""
                inscricao_estadual = ""

                # Iterar pelas linhas para identificar os campos e valores
                i = 0
                while i < len(linhas):
                    linha = linhas[i]
                    try:
                        if "Inscrição Municipal:" in linha:
                            inscricao_municipal = linhas[i + 1].strip()
                            i += 2
                        elif "Inscrição Estadual:" in linha: 
                            inscricao_estadual = linhas[i + 1].strip()   
                            i += 2
                        else:
                            i += 1
                    except IndexError as e:
                        # Log the error or print a warning
                        print(f"Aviso: Não é possível processar a linha {i}. {str(e)}")
                        warning = True
                        warning_message = f"Aviso: Não é possível processar a linha {i}. {str(e)}"
                        
                        break # Exit the loop or continue based on your needs

                if inscricao_municipal == "":
                    inscricao_municipal = "None"        
                if inscricao_estadual == "":
                    inscricao_estadual = "None"

                nf_data_prestador['inscricao_municipal'] = inscricao_municipal
                nf_data_prestador['inscricao_estadual'] = inscricao_estadual


                
            elif frame_father == "2_frame_dados_prestador":
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                tipo = None

                if "\n\n" in extracted_text_frame:
                    linhas = extracted_text_frame.split('\n\n')
                    tipo = 2
                    
                else:
                    linhas = extracted_text_frame.split('\n')    
                    tipo = 1
                
                # Inicializar variáveis para armazenar os valores
                nome_razao_social = None
                nome_fantasia = None
                endereco = None
                email = None

                # Iterar pelas linhas para identificar os campos e valores
                i = 0
                while i < len(linhas):
                    linha = linhas[i]
                    if "Nome/Razão Social:" in linha:
                        if tipo == 2:
                            texto1 = linhas[i + 1].strip()
                            texto2 = texto1.split('\n')
                            nome_razao_social = texto2[i].strip()
                        else:
                            nome_razao_social = linhas[i + 1].strip()   

                        i += 2
                    elif "Nome de Fantasia:" in linha:
                        nome_fantasia = linhas[i + 1].strip()
                        i += 2
                    elif "Endereço:" in linha:
    
                        if tipo == 2:
                            texto3 = linhas[i + 1].strip()
                            texto4 = texto3.split('\n')
                            
                            endereco = texto4[0].strip()
                        else:
                            endereco = linhas[i + 1].strip()      
                        i += 2
                    elif "E-mail:" in linha:
                        email = linhas[i + 1].strip()
                        i += 2
                    else:
                        i += 1

                nf_data_prestador['razao_social'] = nome_razao_social
                nf_data_prestador['nome_fantasia'] = nome_fantasia
                nf_data_prestador['endereco'] = endereco
                nf_data_prestador['email'] = email

    return nf_data_prestador

# 3 - TOMADOR DE SERVIÇO
def processa_tomador():
    nf_data_tomador = {}

    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == modelo]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "3_frame_cnpj_tomador":
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                nf_data_tomador_cnpj = extract_fields_tomador_cnpj(extracted_text_frame)
                
            elif frame_father == "3_frame_inscricao_tomador":
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                nf_data_tomador_inscricao = extract_fields_tomador_inscricao(extracted_text_frame)
            
            elif frame_father == "3_frame_dados_tomador":
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                nf_data_tomador = extract_fields_tomador_dados(extracted_text_frame)
                
    nf_data_tomador.update(nf_data_tomador_cnpj) 
    nf_data_tomador.update(nf_data_tomador_inscricao)
    nf_data_tomador.update(nf_data_tomador)
    
    return nf_data_tomador     

# 4 - DESCRIMINACAO DOS SERVIÇOS
def processa_servico():
    
    nf_data_servico = {}

    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == modelo]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "4_frame_descricao_totais":
                nf_data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                text = extracted_text_frame.replace('\n', ' ')
                label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                if text.startswith(label):
                    text = text[len(label):].strip()
                nf_data_servico['discriminacao_servicos'] = text 
            
    return nf_data_servico

# 5 - VALOR TOTAL
def processa_total():
    
    nf_data_valor_total = {}
    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == modelo]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "4_frame_descricao_totais":
                nf_data_valor_total['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
                # Extrai coordenadas para recorte
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                text = extracted_text_frame.replace('\n', ' ')
                label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                if text.startswith(label):
                    text = text[len(label):].strip()
                nf_data_valor_total['discriminacao_servicos'] = text
                 
    return nf_data_valor_total  

# 6 - CNAE e Item da Lista de Serviços
def processa_cnae_itens():
    
    nf_data_cnae = {}


    # Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
    filtered_frames_info = frames_info[frames_info['model'] == modelo]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        frame_model = row_frame['model']
        if frame_model == model:
            frame_father = row_frame['label']
            # ... Select specific frames ...
            if frame_father == "4_frame_cnae_itens_servico":
                nf_data_cnae['secao'] = "6. CNAE e Item da Lista de Serviços"
                x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
                extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
                
                linhas = extracted_text_frame.split('\n')
                i = len(linhas)
                texto_cnae = linhas[0].replace(' . .', '')
                cnae = re.sub(r'^CNAE - ', '', texto_cnae, count=1)

                # Item da Lista de Serviços 
                texto_novo = linhas[1]
                texto_novo = texto_novo.replace('\n', '')

                item_lista_servico = re.sub(r'^Item da Lista de Serviços - ', '', texto_novo, count=1)

                nf_data_cnae['cnae'] = cnae
                nf_data_cnae['item_lista_servicos'] = item_lista_servico
                
    return nf_data_cnae  

# 7 - VALORES E IMPOSTOS
def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores

# secao: 8 - DADOS COMPLEMENTARES & 10. OBSERVACOES
def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao        
                

In [ ]:
nf_data_tomador

### Funcoes de manejo de documentos

In [ ]:
# move NF processadas ok
def move_pdf_processed_ok(document_path, nf_processada_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(nf_processada_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message

In [ ]:
if model_dict[name_prefeitura]:
    modelo = model_dict[name_prefeitura]
    model = model_dict[name_prefeitura]
    message_log = f"Nome de prefeitura: {name_prefeitura} encontrada para: {doc2convert} modelo a ser utilizado: {modelo}"
    print(message_log)
    #logging.info(message_log) 
else:
    modelo = model_padrao
    message_log = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
    print(message_log)
    #logging.info(message_log)   

## 3. Pipeline

In [ ]:
# 1. Leitura recursiva de diretorios e arquivos a partir de root
#logging.info("Processo iniciado.")
#warning_messages = {}
nf_data_servico = {}
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

for roots, directories, documents in os.walk(root_pdf_path):
    batch_name = os.path.basename(roots)
    source_path = roots
    for document in documents:
        doc2convert = document
        document_path_1 = os.path.join(roots, document)
        message_log = f"Processo iniciado para lote: {directories}"
        #logging.info(message_log)
        print(f'{roots}  | Batch: {batch_name} | doc2convert: {doc2convert} | document_path_1: {document_path_1}\n')
        #logging.info("Processando arquivo: %s", document_path_1)
        message_log = is_pdf_searchable(document_path_1)
        #logging.info(message_log)
        

        
        message_log = f"Processo de extraçao iniciado para documento: {doc2convert}"
        #logging.info(message_log)
        
        root = roots
        
        # 2. Ajusta o nome do arquivo tirando caracteres especiais e a extensao
        doc2convert_named = conv_filename_no_ext(doc2convert)
        
        message_log = f"Processado nome para documento: {doc2convert} -> {doc2convert_named}"
        #logging.info(message_log)
        



        # 3. Converte e resize da imagem "on the fly"
        image_2work, name_image_2work = convertResize(doc2convert_named, document_path_1, image_resized_path) #Utilizado como origem: document_path_1

        #print(name_image_2work)
        message_log = f"Documento {doc2convert} convertido e ajustado para tamanho arquivo: {name_image_2work}"
        #logging.info(message_log) 


        # 4. Busco nome da prefeitura e modelo
        name_prefeitura = pequisaModel(image_2work)

        modelo = model_dict[name_prefeitura]

        if model_dict[name_prefeitura]:
            modelo = model_dict[name_prefeitura]
            model = model_dict[name_prefeitura]
            message_log = f"Nome de prefeitura: {name_prefeitura} encontrada para: {doc2convert} modelo a ser utilizado: {modelo}"
            #logging.info(message_log) 
        else:
            modelo = model_padrao
            message_log = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
            print(message_log)
            #logging.info(message_log)   
        
        nro_nota = None
            
        # secao: 1 - CABECALHO
        data_cabecalho = {}
        data_cabecalho, nro_nota = processa_cabecalho() 
        
        # secao: 2 - PRESTADOR DE SERVIÇO
        data_prestador = {}
        data_prestador = processa_prestador()  
        
        # secao: 3 - TOMADOR DE SERVIÇO
        data_tomador = {}
        data_tomador = processa_tomador()  
        
        # secao: 4 - DESCRIMINACAO DOS SERVIÇOS
        data_servico = {}
        data_servico = processa_servico()
        
        # secao: 5 - VALOR TOTAL
        data_valor_total = {}
        #data_valor_total = processa_total()
        father_value = "4_frame_valor_total"
        section = "5. VALOR TOTAL"
        
        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_valor_total.update(result)
        
        # secao: 6 - CNAE e Item da Lista de Serviços
        data_CNAE = {}
        data_CNAE = processa_cnae_itens()

        # secao: 7 - VALORES E IMPOSTOS & 9 - OUTRAS INFORMAÇOES / CRITICAS
        data_valores = {}
        father_value = "5_frame_valores_impostos"
        section = "7. VALORES E IMPOSTOS"

        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_valores.update(result)
            
        # secao: 8 - DADOS COMPLEMENTARES"
        data_dados_complementares = {}
        f_father = "5_frame_dados_complementares"
        section = "8. DADOS COMPLEMENTARES"

        data_dados_complementares = extract_dados_comple_obs(modelo, f_father, section)                                           
                                
                                
        # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
        data_outras_informacoes = {}
        father_value = "5_frame_inf_criticas"
        section = "9. OUTRAS INFORMAÇOES / CRITICAS"

        result = extract_fields_box(modelo, father_value, section)
        if result:
            data_outras_informacoes.update(result)                        
                            

        # secao: 10. OBSERVACOES
        data_observacao = {}
        f_father = "5_frame_observacao"
        section = "10. OBSERVACOES"

        data_observacao = extract_dados_comple_obs(modelo, f_father, section)
        
        
        #nr_nro_nf = nro_nota
        
        # calling this function    
        success, dest_path, error_message = move_pdf_processed_ok(document_path_1, nf_processada_path, batch_name, doc2convert)
        if success:
            print(f"Sucesso: {dest_path}")
        else:
            print(f"Um erro ocorreu: {error_message}") 
        
            
        nome_arquivo_json = os.path.basename(root) + ".json"
        nome_arquivo = doc2convert
                                    
        pdf_info[nro_nota] = {
            "dados_NF_PDF": {
                "data_cabecalho": data_cabecalho,
                "data_prestador": data_prestador,
                "data_tomador": data_tomador,
                "data_servico": data_servico,
                "data_valor_total": data_valor_total,
                "data_CNAE": data_CNAE,
                "data_valores": data_valores,
                "data_dados_complementares": data_dados_complementares,
                "data_outras_informacoes": data_outras_informacoes,
                "data_observacao": data_observacao,
                },
            "diretorio": os.path.basename(root),
            "nome_arquivo": nome_arquivo,    
        }            
            
                        
# Salvando as informações em um arquivo JSON
json_file_path = os.path.join(json_path, nome_arquivo_json)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")



In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/nota fiscal 37420230815_11212276.pdf.jpg"

In [ ]:
# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

In [ ]:
modelo = "mage_1"

In [ ]:
# To draw everything
draw_box_model(modelo, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


### Chamada de funcoes

In [ ]:
# 2. Ajusta o nome do arquivo tirando caracteres especiais e a extensao
doc2convert_named = conv_filename_no_ext(doc2convert)


# 3. Converte e resize da imagem "on the fly"
image_2work, name_image_2work = convertResize(doc2convert_named, document_path_1, image_resized_path) #Utilizado como origem: document_path_1

#print(name_image_2work) 


# 4. Busco nome da prefeitura e modelo
name_prefeitura = pequisaModel(image_2work)

modelo = model_dict[name_prefeitura]

if model_dict[name_prefeitura]:
    modelo = model_dict[name_prefeitura]
    print(f'\nNome Prefeitura: {name_prefeitura},  modelo: {modelo}') 
else:
    modelo = model_padrao
    message = f'\nmodelo nao encontrado para prefeitura {name_prefeitura}, sera utilizado o modelo {modelo}'   
    print(message)     

## Pipelines Atuais

### Final - tratamento dos arquivos 

In [ ]:
tipo_pdf = "pequisavel"
tipo_pdf = "nao pequisavel"

In [ ]:
# 4.5 Definir o tipo de PDF a buscar  (TESTE APENAS)
#source_directory = root_external_pdf_path

#print(source_directory )
destination_directory = root_pdf_path
if tipo_pdf == "pequisavel":
    source_directory = root_external_pdf_pesquisavel_path
    print(f'\nTipo PDF a utilizar: {tipo_pdf} | source_directory: {source_directory} \n\nDocumento que sera movido: {document_path_1}\n')
    
else:
    source_directory = root_external_pdf_path

#print(f'\nTipo PDF a utilizar: {tipo_pdf} | source_directory: {source_directory} \n\nDocumento que sera movido: {document_path_1}\n')   


# 5. MOVER PDF para diretorio de processados
source_path = document_path_1
destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')

print(f'\nsource_path: {source_path} | destination_path: {destination_path} \n') 


shutil.move(source_path, destination_path)

# 6. ***DELETAR**** o arquivo de imagem gerado
image_path_to_delete = name_image_2work
os.remove(image_path_to_delete)


# Verifique se o diretório de destino está vazio
if os.listdir(destination_directory):
    raise Exception("O diretório de destino não está vazio!")

for roots, directories, documents in os.walk(source_directory):
    # Filtre os documentos para incluir apenas aqueles com extensão .pdf
    pdf_files = [doc for doc in documents if doc.lower().endswith('.pdf')]
    
    if pdf_files: # Verifique se há algum arquivo PDF no diretório
        first_pdf_file = pdf_files[0] # Obtenha o primeiro arquivo PDF
        source_path = os.path.join(roots, first_pdf_file) # Construa o caminho completo para o primeiro arquivo PDF
        print(f"Found the first PDF file: {source_path}")
        
        # Mova o arquivo para o diretório de destino
        shutil.move(source_path, destination_directory)
        
        break # Saia do loop após encontrar o primeiro arquivo PDF

# Outros itens do Processo

### REVISAR

In [ ]:
               # Iterar pelas linhas para identificar os campos e valores
                i = 0
                while i < len(linhas):
                    linha = linhas[i]
                    if "Inscrição Municipal:" in linha:
                        inscricao_municipal = linhas[i + 1].strip()
                        i += 2
                    elif "Inscrição Estadual:" in linha: 
                        inscricao_estadual = linhas[i + 1].strip()   
                        i += 2
                    else:
                        i += 1
                        
                if inscricao_municipal == "":
                    inscricao_municipal = "None"        
                if inscricao_estadual == "":
                    inscricao_estadual = "None"

                nf_data_prestador['inscricao_municipal'] = inscricao_municipal
                nf_data_prestador['inscricao_estadual'] = inscricao_estadual

### Modelos de pipelines utilizados

In [ ]:
for root, dirs, files in os.walk(root_pdf_path):
    print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        #if file.lower().endswith('.pdf'):

In [ ]:
# Modelo iniciado com funçoes do model

nf_model = "mage_1"

nf_data_servico = {}

pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

for root, dirs, files in os.walk(root_directory):
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.pdf'):
            #print(file_path)
                
            # 1. Conversao para imagem e resize
            converte2image(file_path)
                
            # 2. Resize
            resizeImage(image_src)
                
            # 3. Executar cortes
            #file_path = img_resi_path
            #image_to_crop = Image.open(img_resi_path).convert("RGB")
            #width, height = image_to_crop.size

            cropSections(nf_model, img_resi_path)
                
            for roots, directories, images in os.walk(root_dir_section_images):
                for image in images:
                    #print(f'\n{roots}, {directories}, {images}')
                    print(image)
                    image_path = os.path.join(roots, image)
                    #print(image_path, image)
                    frame = image
                    if frame == "0_frame_dados_nf.jpg":
                        nro_nota = 0
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_dados, nro_nota = extract_fields_dados(text)

                    if frame == "0_frame_prefeitura_nf.jpg":
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prefeitura, nome_prefeitura  = extract_fields_prefeitura(text)    
                        
                    if frame == "1_frame_prestador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)
                        
                               
                    if frame == "1_frame_prestador_inscricao.jpgg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)    
                            
                    if frame == "1_frame_prestador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_prestador = extract_fields_prestador(text)      
                        
                        
                        
                        
                        
                        
                        
                    if frame == "2_frame_tomador_cnpj.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_cnpj = extract_fields_tomador_cnpj(text)                                                       
                    
                    
                    if frame == "2_frame_tomador_inscricao.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador_inscricao = extract_fields_tomador_inscricao(text)                            
                        
                        
                    if frame == "2_frame_tomador_servico.jpg": 
                        image_to_ocr = Image.open(image_path).convert('RGB')
                        text = pytesseract.image_to_string(image_to_ocr, lang='por', config=tessdata_dir_config)
                        nf_data_tomador = extract_fields_tomador(text)  
                            
                            
                            
                            # 6. Processa analise de tabela
                            #results = processaTabela(image_path)
                            
                            # 7. Processa OCR
                            #dados_da_tabela = processaOcrTable(image_path, results)
                            
                            #for i in dados_da_tabela:
                                #if dados_da_tabela[i]['label'] == "table column":
                                    #text = dados_da_tabela[i]['texto']
                                    #print(f"\n{i}: \n{dados_da_tabela[i]['texto']}")
                                    
                    nr_nro_nf = nro_nota
                        
                    nome_arquivo_json = os.path.basename(root) + ".json"
                    nome_arquivo = file
                                                
                    pdf_info[nr_nro_nf] = {
                        "dados_NF_PDF": {
                            "data_nf": nf_data_dados,
                            "data_prefeitura": nf_data_prefeitura,
                            "data_prestador": nf_data_prestador,
                            "data_tomador_cnpj": nf_data_tomador_cnpj,
                            "data_tomador_inscricao": nf_data_tomador_inscricao,
                            "data_tomador": nf_data_tomador,
                            },
                        "diretorio": os.path.basename(root),
                        "nome_arquivo": nome_arquivo,    
                    }            
                        
                        
# Salvando as informações em um arquivo JSON
json_file_path = os.path.join(target_directory, nome_arquivo_json)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")

### Chamada das funcoes

In [ ]:
# secao: 1 - CABECALHO
data_cabecalho = {}
data_cabecalho = processa_cabecalho()

In [ ]:
# secao: 2 - PRESTADOR DE SERVIÇO
data_prestador = {}
data_prestador = processa_prestador()

In [ ]:
# secao: 3 - TOMADOR DE SERVIÇO
data_tomador = {}
data_tomador = processa_tomador()

In [ ]:
# secao: 4 - DESCRIMINACAO DOS SERVIÇOS
data_servico = {}
data_servico = processa_servico()

In [ ]:
# secao: 5 - VALOR TOTAL
data_valor_total = {}
data_valor_total = processa_total()

In [ ]:
# secao: 6 - CNAE e Item da Lista de Serviços
data_CNAE = {}
data_CNAE = processa_cnae_itens()

In [ ]:
# secao: 7 - VALORES E IMPOSTOS & 9 - OUTRAS INFORMAÇOES / CRITICAS
data_valores = {}
father_value = "5_frame_valores_impostos"
section = "7. VALORES E IMPOSTOS"

result = extract_fields_box(modelo, father_value, section)
if result:
    data_valores.update(result)  

In [ ]:
# secao: 8 - DADOS COMPLEMENTARES"
data_dados_complementares = {}
f_father = "5_frame_dados_complementares"
section = "8. DADOS COMPLEMENTARES"

data_dados_complementares = extract_dados_comple_obs(modelo, f_father, section)

In [ ]:
# secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
data_outras_informacoes = {}
father_value = "5_frame_inf_criticas"
section = "9. OUTRAS INFORMAÇOES / CRITICAS"

result = extract_fields_box(modelo, father_value, section)
if result:
    data_outras_informacoes.update(result)

In [ ]:
# secao: 10. OBSERVACOES
data_observacao = {}
f_father = "5_frame_observacao"
section = "10. OBSERVACOES"

data_observacao = extract_dados_comple_obs(modelo, f_father, section)

### Draw Boxes

In [ ]:
# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw everything
draw_box_model(modelo, boundaries_info, sections_info, frames_info, field_boxes_info)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


### Pipeline reserva

In [ ]:
# Modelo anterior do pipeline

nf_data_servico = {}

pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

for root, dirs, files in os.walk(root_directory):
    for file in files:
        file_path = os.path.join(root, file)
        
        if file.lower().endswith('.pdf'):
            if is_pdf_searchable(file_path):
                
                status = "O PDF é pesquisável"
                nro_nota = 0
                nr_nro_nf = 0
                
                #Definindo a pagina
                # Carregar o arquivo PDF
                pdf_document = fitz.open(file_path)
                # Página do PDF
                page_number = 0  # Defina o número da página que deseja analisar
                page = pdf_document[page_number]

                # Definir retângulo de interesse
                x0 = 0
                y0 = 4
                x1 = 600
                y1 = 200  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                if text:
                    page_number = 0
                else:
                    page_number = 1
                
                
                # 1 - cabecalho
                #pdf_document = fitz.open(file_path)
                #page_number = 0  # Defina o número da página que deseja analisar
                page = pdf_document[page_number]
                x0 = 0
                y0 = 0
                x1 = 600
                y1 = 110
                
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                nf_data_cabecalho, nro_nota = extract_fields_cabecalho(text)
                
                
                # 2. PRESTADOR DE SERVIÇO
                # Definir retângulo de interesse
                x0 = 0
                y0 = 100
                x1 = 600
                y1 = 236  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                
                nf_data_prestador = extract_fields_prestador(text)
                
                # 3. TOMADOR DE SERVIÇO
                # Definir retângulo de interesse
                x0 = 0
                y0 = 210
                x1 = 600
                y1 = 340  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                nf_data_tomador = extract_fields_tomador(text)
                
                
                # 4. DESCRIMINACAO DOS SERVIÇOS
                nf_data_servico = {}
                nf_data_servico['secao'] = "4. DESCRIMINACAO DOS SERVIÇOS"
                # Definir retângulo de interesse
                x0 = 0
                y0 = 330
                x1 = 600
                y1 = 500  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                # Remover quebras de linha e rótulo
                text = text.replace('\n', ' ')
                label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                if text.startswith(label):
                    text = text[len(label):].strip()

                # Atribuir texto ao dicionário
                nf_data_servico['discriminacao_servicos'] = text
                
                
                # 5. VALOR TOTAL
                nf_data_valor_total = {}
                nf_data_valor_total['secao'] = "5. VALOR TOTAL"

                # Definir retângulo de interesse
                x0 = 0
                y0 = 500
                x1 = 600
                y1 = 535  # Ajuste este valor para delimitar a região vertical


                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                # Usar expressão regular para extrair apenas os caracteres numéricos e pontos decimais
                valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                if valor_total_match:
                    valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                    nf_data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                    
                    
                
                
                # 6. CNAE e Item da Lista de Serviços
                nf_data_CNAE = {}
                nf_data_CNAE['Secao'] = "6. CNAE e Item da Lista de Serviços"

                # Definir retângulo de interesse CNAE
                x0 = 0
                y0 = 530
                x1 = 600
                y1 = 540  # Ajuste este valor para delimitar a região vertical


                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))


                # Extrair CNAE
                nf_data_CNAE_match = re.search(r'CNAE\s+(.+)', text)
                if nf_data_CNAE_match:
                    # Remove a primeira ocorrência de "CNAE:"
                    nf_data_CNAE_str = re.sub(r'^CNAE - ', '', text, count=1)
                    # Remover quebras de linha
                    nf_data_CNAE_str = nf_data_CNAE_str.replace('\n', ' ')
                    nf_data_CNAE['cnae'] = nf_data_CNAE_str
                    
                
                
                # Item da Lista de Serviços    
                # Definir retângulo de interesse - Item da Lista de Serviços
                x0 = 0
                y0 = 545
                x1 = 600
                y1 = 560  # Ajuste este valor para delimitar a região vertical    
                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))     
                    
                # Extrair Item da Lista de Serviços
                nf_item_lista_servicos_match = re.search(r'Item da Lista de Serviços\s+(.+)', text)
                if nf_item_lista_servicos_match:
                    nf_item_lista_servicos_str = re.sub(r'^Item da Lista de Serviços - ', '', text, count=1) 
                    # Remover quebras de linha
                    #nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n \n', '')
                    nf_item_lista_servicos_str = nf_item_lista_servicos_str.replace('\n', ' ')
                    nf_data_CNAE['item_lista_servicos'] = nf_item_lista_servicos_str
                      
                    
                    
                    
                    
                
                
                # 7. VALORES E IMPOSTOS
                # Definir retângulo de interesse
                x0 = 0
                y0 = 550
                x1 = 600
                y1 = 650  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                # Extrair campos e valores
                nf_data_valores = extract_fields_impostos(text)
                
                # 8. DADOS COMPLEMENTARES
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['secao'] = "8. DADOS COMPLEMENTARES"

                # Definir retângulo de interesse
                x0 = 0
                y0 = 650
                x1 = 600
                y1 = 680  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', text, count=1)
                if text == " ":
                    text = "NONE"
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                
                
                
                # 9. OUTRAS INFORMAÇOES / CRITICAS  
                # Definir retângulo de interesse
                x0 = 0
                y0 = 680
                x1 = 600
                y1 = 725  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                # Extrair campos e valores
                nf_data_outras_informacoes = extract_fields_outras_info(text)
                
                
                # 10. OBSERVACOES
                nf_data_observacao = {}
                nf_data_observacao['secao'] = "10. OBSERVACOES"
                # Definir retângulo de interesse
                x0 = 0
                y0 = 725
                x1 = 600
                y1 = 760  # Ajuste este valor para delimitar a região vertical

                # Extrair texto dentro do retângulo
                text = page.get_text("text", clip=(x0, y0, x1, y1))

                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', text, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                nr_nro_nf = nro_nota
                
                nome_arquivo_json = os.path.basename(root) + ".json"
                
                #json_file_path = os.path.join(target_directory, diretorio, ".json")
                
                nome_arquivo = file
                #pdf_info["diretorio"] = os.path.basename(root)
                pdf_info[nr_nro_nf] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": nf_data_cabecalho,
                        "data_prestador": nf_data_prestador,
                        "data_tomador": nf_data_tomador,
                        "data_servico": nf_data_servico,
                        "data_valor_total": nf_data_valor_total,
                        "data_CNAE": nf_data_CNAE,
                        "data_valores": nf_data_valores,
                        "data_dados_complementares": nf_data_dados_complementares,
                        "data_outras_informacoes": nf_data_outras_informacoes,
                        "data_observacao": nf_data_observacao,
                    },
                    "diretorio": os.path.basename(root),
                    "nome_arquivo": nome_arquivo,
                }
                
                
                pdf_document.close()

# Salvando as informações em um arquivo JSON
json_file_path = os.path.join(target_directory, nome_arquivo_json)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")

### Chamada das funçoes

### Modelo de consistencia de cnae

In [ ]:
# Assume extracted_cnae contains the extracted CNAE code or description from the document
extracted_cnae = 'some_value'

# Search the DataFrame for the extracted CNAE information (adjust the column name as needed)
matching_row = cnae_df[cnae_df['cnae_column'] == extracted_cnae]

# Use the values from the matching row
if not matching_row.empty:
    whole_value = matching_row.iloc[0] # Adjust to get the specific value you need
    # Process the whole_value as needed
else:
    print('CNAE not found in repository')

In [ ]:
mage_cnae_x_item_servico_df.head()

In [ ]:
item_servico_dict[7.02].upper()
cnae_dict[4313400].upper()

### Template de funçao: frames_info -> sframe_fields_info

In [ ]:
def extract_fields_frame(modelo, father_value, section):
    

    frame_label = father_value
    print(section)
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_label) & (frames_info['model'] == modelo)]
    
    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == father_value) & (sframe_fields_info['model'] == modelo)]
    
    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        #extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        #value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        #value = value.strip()
        
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            
            print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))


### Outras possiveis chamadas de funcao mais inteligente

- data_prestador

- data_CNAE

In [ ]:
# data_prestador
data_prestador = {}
father_value = "2_frame_cnpj_prestador"
section = "2. PRESTADOR DE SERVIÇO"


extract_fields_frame(modelo, father_value, section)

In [ ]:
# data_CNAE
data_CNAE = {}
father_value = "4_frame_cnae_itens_servico"
section = "6. CNAE e Item da Lista de Serviços"


extract_fields_frame(modelo, father_value, section)

## 5. Informacoes dos DFSs

In [ ]:
# Informaçao sobre os DF sendo utilizados


sections_info

frames_info.head()

sframe_fields_info.head()

field_boxes_info.head()

In [ ]:
frames_info.head()

In [ ]:
sframe_fields_info.head()

In [ ]:
field_boxes_info.head()

### Filtros

In [ ]:
# 1. Filtrando o frames_info para buscar os dados de corte
filtered_frames_info = frames_info[(frames_info['label'] == father_value) & (frames_info['model'] == modelo)]

In [ ]:
# 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == father_value) & (sframe_fields_info['model'] == modelo)]

In [ ]:
# 3. Filtrando o field_boxes_info para buscar os dados de corte dos campos com BOX
filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]

## 6. Operaçoes com arquivos e diretorios & Logging

<mark> Config </mark>
- root_pdf_path  (local inicial dos PDFs)

- image_resized_path (local da imagem que foi convetida e resized)

- nf_processada_path (local onde ficam os PDFs processados)


<mark> Criados no processo </mark>

- image_2work (arquivo de imagem resized criado)

- name_image_2work (path e nome da imagem resized criada)

- document_path (path e nome do PDF utilizado para tratamento)

In [ ]:
def move_pdf_processed_ok(document_path):
    
    source_path = document_path
    destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
    shutil.move(source_path, destination_path)
    

In [ ]:
# mover PDF para diretorio de processados
source_path = document_path
destination_path = os.path.join(f'{nf_processada_path}/{str(doc2convert)}')
shutil.move(source_path, destination_path)

In [ ]:
# ***DELETAR**** o arquivo de imagem gerado
image_path_to_delete = name_image_2work
os.remove(image_path_to_delete)

In [ ]:
# voltar (mover) PDF para diretorio para ser processado
source_path = destination_path

destination_path = document_path

shutil.move(source_path, destination_path)

In [ ]:
# Processo para iterar estrutura de arquivos, diretorios
for roots, directories, documents in os.walk(image_resized_path):
    print(f'\n {documents} {name_image_2work}\n\n')
    #if documents == :
        

In [ ]:
# 7. Buscar o primeiro documento do diretorio EXTERNO para processar
source_directory = root_external_pdf_path
destination_directory = root_pdf_path

# Verifique se o diretório de destino está vazio
if os.listdir(destination_directory):
    raise Exception("O diretório de destino não está vazio!")

for roots, directories, documents in os.walk(source_directory):
    # Filtre os documentos para incluir apenas aqueles com extensão .pdf
    pdf_files = [doc for doc in documents if doc.lower().endswith('.pdf')]
    
    if pdf_files: # Verifique se há algum arquivo PDF no diretório
        first_pdf_file = pdf_files[0] # Obtenha o primeiro arquivo PDF
        source_path = os.path.join(roots, first_pdf_file) # Construa o caminho completo para o primeiro arquivo PDF
        print(f"Found the first PDF file: {source_path}")
        
        # Mova o arquivo para o diretório de destino
        shutil.move(source_path, destination_directory)
        
        break # Saia do loop após encontrar o primeiro arquivo PDF

### Logging

LOG - Single File vs. Separate Files

1. Single File vs. Separate Files
Single File: Suitable if the volume of logs is manageable and you want a consolidated view of all activities.
Separate Files: Useful if you want to organize logs by day or by a specific module or process. This can help in troubleshooting specific issues and managing large volumes of logs.
2. Concise vs. Detailed Logs
You can create different log levels to handle concise and detailed logging:

Concise Log: Use the INFO level to log only essential information, such as process start and end times, success messages, etc.
Detailed Log: Use the DEBUG level to log detailed information, like variable values, intermediate steps, etc.
Example with Python's logging Module
Here's an example of how you might set up logging to handle both separate files and different log levels:

In [ ]:
import logging

# Configure logging
log_file_path = 'novo_modelo/execucao_pipeline_extracao_pdf.log'
logging.basicConfig(
    filename=log_file_path,
    filemode='a', # Append mode
    level=logging.INFO, # Log level (e.g., INFO, WARNING, ERROR)
    format='%(asctime)s - %(levelname)s - %(message)s', # Log message format
    datefmt='%Y-%m-%d %H:%M:%S' # Date format
)

In [ ]:
# Create a logger
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)

# Create handlers
info_handler = logging.FileHandler('info.log')
info_handler.setLevel(logging.INFO)

error_handler = logging.FileHandler('error.log')
error_handler.setLevel(logging.ERROR)

# Create a formatter and set it for the handlers
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
info_handler.setFormatter(formatter)
error_handler.setFormatter(formatter)

# Add the handlers to the logger
logger.addHandler(info_handler)
logger.addHandler(error_handler)

# Log messages
logger.info('This is an info message.')
logger.error('This is an error message.')


In [ ]:

# Example of concise log (INFO level)
logging.info("Processo iniciado.")

# Example of detailed log (DEBUG level)
logging.debug("Variable x value is 10.")

# ... rest of your code ...

# Handle exceptions with logging
try:
    # Your code that might raise an exception
    pass
except Exception as e:
    logging.error(f"An error occurred: {e}")

In [ ]:
import logging

# Configure logging
log_file_path = 'novo_modelo/execucao_pipeline_extracao_pdf.log'
logging.basicConfig(
    filename=log_file_path,
    filemode='a', # Append mode
    level=logging.INFO, # Log level (e.g., INFO, WARNING, ERROR)
    format='%(asctime)s - %(levelname)s - %(message)s', # Log message format
    datefmt='%Y-%m-%d %H:%M:%S' # Date format
)

In [ ]:
logging.info("Pipeline iniciado.")

In [ ]:
# Example usage of the logger
logging.info("Pipeline started.")
logging.info("Processing file: %s", first_pdf_file_path)
logging.warning("File has more than one page.")
logging.error("An error occurred while processing file: %s", first_pdf_file_path)

In [ ]:
logging.info("Pipeline iniciado.")

In [ ]:
# Example usage of the logger
logging.info("Pipeline started.")
logging.info("Processing file: %s", first_pdf_file_path)
logging.warning("File has more than one page.")
logging.error("An error occurred while processing file: %s", first_pdf_file_path)

By changing the log level, you can control the verbosity of the logs. If you want separate log files for each day, the above code will create a new file every day based on the date.

Remember, logging strategy should align with your application's needs, including debugging, auditing, and compliance requirements. Make sure to review and test your logging setup to ensure it meets your needs.

In [ ]:
import logging
import datetime

# Get the current date for filename
current_date = datetime.datetime.now().strftime("%Y-%m-%d")
log_filename = f"log_{current_date}.txt"

# Set up logging
logging.basicConfig(
    filename=log_filename,
    level=logging.DEBUG,  # Log everything (change to INFO for concise logs)
    format="%(asctime)s - %(levelname)s - %(message)s",
)

# Example of concise log (INFO level)
logging.info("Processing started.")

# Example of detailed log (DEBUG level)
logging.debug("Variable x value is 10.")

# ... rest of your code ...

# Handle exceptions with logging
try:
    # Your code that might raise an exception
    pass
except Exception as e:
    logging.error(f"An error occurred: {e}")


## 7. Teste e codigos para analisar

### Funcoes de melhoria

In [ ]:
    
#Funçao de formatacao de numeros
def format_number(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para campos %
    return float(number_str)




In [ ]:
def extract_dados_cabecalho(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_frame = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        values = extracted_text_frame.split('\n')
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            nf_data_cabecalho_prefeitura = {}
            nf_data_cabecalho_prefeitura['secao'] = section
            type_value = row_field['type']
            label_value = row_field['label']
            reference_value = row_field['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    nf_data_cabecalho_prefeitura.update(result)
            
        return nf_data_cabecalho_prefeitura   

In [ ]:
# data_cabecalho
f_father = "1_frame_prefeitura_nf"
section = "1 - CABECALHO"

#dict consolidador
data_cabecalho = {}

data_cabecalho = extract_dados_cabecalho(modelo, f_father, section)

### Tesseract

<mark> Testar esta configuraçao de Tesseract </mark>

In [ ]:
def detect(cropped_frame, is_number = False):
    if (is_number):
        text = pytesseract.image_to_string(cropped_frame,
                                           config ='-c tessedit_char_whitelist=0123456789 --psm 10 --oem 2')
    else:
        text = pytesseract.image_to_string(cropped_frame, config='--psm 10')        
        
    return text


### Draw Boxes with matplotlib

Using the Matplotlib library to draw boxes on an image is entirely feasible and provides more control over the appearance and customization of the drawings. Below, I'll provide an example of how you can draw boxes on an image using Matplotlib.

First, make sure you have the necessary libraries installed:

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_boxes(image, boxes_info, color='r'):
    # Create a figure and axis
    fig, ax = plt.subplots(1)
    
    # Display the image
    ax.imshow(image)

    # Draw the boxes
    for index, row in boxes_info.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
        # Create a Rectangle patch
        rect = patches.Rectangle((x0, y0), x1 - x0, y1 - y0, linewidth=1, edgecolor=color, facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)

    plt.show()


You can call this function with your image object and DataFrame containing the coordinates of the boxes to be drawn:

In [ ]:
draw_boxes(image, boundaries_info, color='r')
draw_boxes(image, sections_info, color='g')


You can customize the appearance of the boxes by changing the parameters of the patches.Rectangle function, such as line width, edge color, and face color.

Using Matplotlib in this way allows you to have more control over the appearance of the plot, and you can easily combine it with other Matplotlib features to create complex visualizations.


### Tecnicas de manitpulacao de imagens

In [ ]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

def convert_pdf_to_string(file_path):

	output_string = StringIO()
	with open(file_path, 'rb') as in_file:
	    parser = PDFParser(in_file)
	    doc = PDFDocument(parser)
	    rsrcmgr = PDFResourceManager()
	    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
	    interpreter = PDFPageInterpreter(rsrcmgr, device)
	    for page in PDFPage.create_pages(doc):
	        interpreter.process_page(page)

	return(output_string.getvalue())


2407-106 TERE SÃO PEDRO.pdf
                
def convert_title_to_filename(title):
    filename = title.lower()
    filename = filename.replace(' ', '_')
    return filename


def split_to_title_and_pagenum(table_of_contents_entry):
    title_and_pagenum = table_of_contents_entry.strip()
    
    title = None
    pagenum = None
    
    if len(title_and_pagenum) > 0:
        if title_and_pagenum[-1].isdigit():
            i = -2
            while title_and_pagenum[i].isdigit():
                i -= 1

            title = title_and_pagenum[:i].strip()
            pagenum = int(title_and_pagenum[i:].strip())
        
    return title, pagenum

In [ ]:
import io
import os
import sys
import time

import fitz
import PySimpleGUI as sg

print(fitz.__doc__)

if not tuple(map(int, fitz.version[0].split("."))) >= (1, 18, 18):
    raise SystemExit("require PyMuPDF v1.18.18+")

dimlimit = 0  # 100  # each image side must be greater than this
relsize = 0  # 0.05  # image : image size ratio must be larger than this (5%)
abssize = 0  # 2048  # absolute image size limit 2 KB: ignore if smaller
imgdir = "output"  # found images are stored in this subfolder

if not os.path.exists(imgdir):  # make subfolder if necessary
    os.mkdir(imgdir)


def recoverpix(doc, item):
    xref = item[0]  # xref of PDF image
    smask = item[1]  # xref of its /SMask

    # special case: /SMask or /Mask exists
    if smask > 0:
        pix0 = fitz.Pixmap(doc.extract_image(xref)["image"])
        if pix0.alpha:  # catch irregular situation
            pix0 = fitz.Pixmap(pix0, 0)  # remove alpha channel
        mask = fitz.Pixmap(doc.extract_image(smask)["image"])

        try:
            pix = fitz.Pixmap(pix0, mask)
        except:  # fallback to original base image in case of problems
            pix = fitz.Pixmap(doc.extract_image(xref)["image"])

        if pix0.n > 3:
            ext = "pam"
        else:
            ext = "png"

        return {  # create dictionary expected by caller
            "ext": ext,
            "colorspace": pix.colorspace.n,
            "image": pix.tobytes(ext),
        }

    # special case: /ColorSpace definition exists
    # to be sure, we convert these cases to RGB PNG images
    if "/ColorSpace" in doc.xref_object(xref, compressed=True):
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        return {  # create dictionary expected by caller
            "ext": "png",
            "colorspace": 3,
            "image": pix.tobytes("png"),
        }
    return doc.extract_image(xref)


fname = sys.argv[1] if len(sys.argv) == 2 else None
if not fname:
    fname = sg.PopupGetFile("Select file:", title="PyMuPDF PDF Image Extraction")
if not fname:
    raise SystemExit()

t0 = time.time()
doc = fitz.open(fname)

page_count = doc.page_count  # number of pages

xreflist = []
imglist = []
for pno in range(page_count):
    sg.QuickMeter(
        "Extract Images",  # show our progress
        pno + 1,
        page_count,
        "*** Scanning Pages ***",
    )

    il = doc.get_page_images(pno)
    imglist.extend([x[0] for x in il])
    for img in il:
        xref = img[0]
        if xref in xreflist:
            continue
        width = img[2]
        height = img[3]
        if min(width, height) <= dimlimit:
            continue
        image = recoverpix(doc, img)
        n = image["colorspace"]
        imgdata = image["image"]

        if len(imgdata) <= abssize:
            continue
        if len(imgdata) / (width * height * n) <= relsize:
            continue

        imgfile = os.path.join(imgdir, "img%05i.%s" % (xref, image["ext"]))
        fout = open(imgfile, "wb")
        fout.write(imgdata)
        fout.close()
        xreflist.append(xref)

t1 = time.time()
imglist = list(set(imglist))
print(len(set(imglist)), "images in total")
print(len(xreflist), "images extracted")
print("total time %g sec" % (t1 - t0))

In [ ]:
"""
A basic text-to-PDF converter
--------------------------------------------------------------------------------
License: GNU GPL V3
(c) 2018 Jorj X. McKie

Usage
-----
python convert.py input.txt
"""

import sys
import fitz

assert len(sys.argv) == 2, "usage: python %s text.file" % (sys.argv[0],)
ifn = sys.argv[1]
ofn = "output1.pdf"

width, height = fitz.paper_size("a4")
fontsz = 10
lineheight = fontsz * 1.2

nlines = int((height - 108.0) / lineheight)

sourcefile = open(ifn)
line_ctr = 0  # page line counter
total_ctr = 0  # total line counter
out_ctr = 0  # count output lines
out_buf = ""  # text of one page

doc = fitz.open()


def page_out(b):
    page = doc.new_page(width=width, height=height)
    return page.insert_text(
        (50, 72),
        text=b,
        fontsize=fontsz,
    )


while True:
    line = sourcefile.readline()
    if line == "":
        break
    out_buf += line
    line_ctr += 1
    total_ctr += 1
    if line_ctr == nlines:
        out_ctr += page_out(out_buf)
        out_buf = ""
        line_ctr = 0

if len(out_buf) > 0:
    out_ctr += page_out(out_buf)

print("PDF conversion results for file '%s':" % (ifn,))
print(out_ctr, "lines read,", total_ctr, "lines written,", nlines, "lines per page.")
print(ofn, "contains", len(doc), "pages.")

# Now add a header and footer to each page
hdr_fontsz = 16
ftr_fontsz = 8
blue = fitz.pdfcolor["blue"]
pspace = 500

for page in doc:
    footer = "%i (%i)" % (page.number + 1, len(doc))  # footer text
    plen_ftr = fitz.get_text_length(footer, fontname="Helvetica", fontsize=ftr_fontsz)
    page.insert_text(
        (50, 50), ifn, color=blue, fontsize=hdr_fontsz  # header = input filename
    )
    page.draw_line(
        fitz.Point(50, 60),
        fitz.Point(50 + pspace, 60),  # line below hdr
        color=blue,
        width=0.5,
    )
    page.draw_line(
        fitz.Point(50, height - 33),  # line above footer
        fitz.Point(50 + pspace, height - 33),
        color=blue,
        width=0.5,
    )
    page.insert_text(
        (50 + pspace - plen_ftr, height - 33 + ftr_fontsz * 1.2),  # insert footer
        footer,
        fontsize=ftr_fontsz,
        color=blue,
    )
    page.clean_contents()

doc.set_metadata(
    {
        "creationDate": fitz.get_pdf_now(),
        "modDate": fitz.get_pdf_now(),
        "creator": "convert.py",
        "producer": "PyMuPDF %s" % fitz.VersionBind,
        "title": "Content of file " + ifn,
        "subject": "Demonstrate methods new_page, insert_text and draw_line",
        "author": "Jorj McKie",
    }
)
doc.subset_fonts()
doc.ez_save(ofn, garbage=4, pretty=True)
doc.close()


### Rotinas anteriores - email

In [ ]:
import os
import zipfile





# Obtém o nome base do arquivo ZIP para usar como subdiretório
zip_basename = os.path.splitext(os.path.basename(zip_file_path))[0]

# Cria o subdiretório com base no nome do arquivo ZIP
root_output_dir = os.path.join(output_dir, zip_basename)
if not os.path.exists(root_output_dir):
    os.makedirs(root_output_dir)

# Dicionário para guardar o nome da pasta e os arquivos associados
folder_file_dict = {}

# Abre o arquivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for member in zip_ref.namelist():
        # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
        parts = member.rsplit('\\', 1)
        folder_name = parts[0] if len(parts) > 1 else ''
        filename = parts[-1]

        if filename:  # ignora diretórios
            # Adiciona ao dicionário
            folder_file_dict.setdefault(folder_name, []).append(filename)

            # Cria um subdiretório se ele não existir
            sub_dir = os.path.join(root_output_dir, folder_name)
            if not os.path.exists(sub_dir):
                os.makedirs(sub_dir)

            # Salva o arquivo no subdiretório especificado
            source = zip_ref.open(member)
            target_path = os.path.join(sub_dir, filename)
            with open(target_path, "wb") as target:
                target.write(source.read())

# Imprime o dicionário
for folder, files in folder_file_dict.items():
    print(f"Pasta: {folder}, Arquivos: {files}")

print(f"Arquivos extraídos para {root_output_dir}")


In [ ]:
import os
from outlook_msg import Message

# Caminho da pasta onde você quer salvar os anexos
pasta_destino = msg_attachment_zip

# Verifica se a pasta existe; se não, cria ela
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

with open(msg_file_path) as msg_file:
    msg = Message(msg_file)

# Contents are the plaintext body of the email
contents = msg.body

# Attachments can be read and saved like so
first_attachment = msg.attachments[0]

# Constrói o caminho completo para o arquivo anexo
caminho_completo_anexo = os.path.join(pasta_destino, first_attachment.filename)

with first_attachment.open() as attachment_fp, open(caminho_completo_anexo, 'wb') as output_fp:
    output_fp.write(attachment_fp.read())

In [ ]:
root_zip = 'pipeline_extracao_documentos/emails/emails_documentos_recebidos/attachments/arquivos_compactados'

In [ ]:
# Diretório onde você quer salvar os arquivos extraídos
output_dir = 'pipeline_extracao_documentos/emails/emails_documentos_recebidos/documentos'

for root, dirs, files in os.walk(root_zip):
    #print(f'{root}  | {dirs} | {document} | {files}\n')
    
    for file in files:
        file_path = os.path.join(root, file)

        if file.lower().endswith('.zip'):
            zip_file = file
            zip_file_path = os.path.join(root, file)
            
            # Dicionário para guardar o nome da pasta e os arquivos associados
            folder_file_dict = {}

            # Abre o arquivo ZIP
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                for member in zip_ref.namelist():
                    # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
                    parts = member.rsplit('\\', 1)
                    folder_name = parts[0] if len(parts) > 1 else ''
                    filename = parts[-1]
                    #iter_output_dir = os.path.join(output_dir, filename)
                    
                    iter_output_dir = os.path.join(output_dir, folder_name)
                    
                    print("Nome da pasta no zip: ", iter_output_dir)
                    print()
                    print("Nome do arquivo na pasta: ", filename)
                    
                    
                    if not os.path.exists(iter_output_dir):
                        os.makedirs(iter_output_dir)

                    if filename:  # ignora diretórios
                        # Adiciona ao dicionário
                        folder_file_dict.setdefault(folder_name, []).append(filename)

                        # Salva o arquivo no diretório especificado
                        source = zip_ref.open(member)
                        target_path = os.path.join(iter_output_dir, filename)
                        with open(target_path, "wb") as target:
                            target.write(source.read())

            # Imprime o dicionário
            for folder, files in folder_file_dict.items():
                print(f"Pasta: {folder}, Arquivos: {files}")

            print(f"Arquivos extraídos para {output_dir}")

In [ ]:
# Extracao do ZIP

# Caminho do arquivo ZIP
zip_file_path = 'pipeline_extracao_documentos/emails/emails_documentos_recebidos/attachments/arquivos_compactados/SAO PEDRO DA ALDEIA_PDF_17282023_1532.zip'

# Path para area ZIP
zip_dir_path = 'pipeline_extracao_documentos/emails/emails_documentos_recebidos/attachments/arquivos_compactados'

# Diretório onde você quer salvar os arquivos extraídos
output_dir = 'pipeline_extracao_documentos/emails/emails_documentos_recebidos/documentos'

# Verifica se o diretório de destino existe; se não, cria um
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dicionário para guardar o nome da pasta e os arquivos associados
folder_file_dict = {}

# Abre o arquivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    for member in zip_ref.namelist():
        # Separa o nome da pasta e o nome do arquivo usando barra invertida como delimitador
        parts = member.rsplit('\\', 1)
        folder_name = parts[0] if len(parts) > 1 else ''
        filename = parts[-1]

        if filename:  # ignora diretórios
            # Adiciona ao dicionário
            folder_file_dict.setdefault(folder_name, []).append(filename)

            # Salva o arquivo no diretório especificado
            source = zip_ref.open(member)
            target_path = os.path.join(output_dir, filename)
            with open(target_path, "wb") as target:
                target.write(source.read())

# Imprime o dicionário
for folder, files in folder_file_dict.items():
    print(f"Pasta: {folder}, Arquivos: {files}")

print(f"Arquivos extraídos para {output_dir}")


In [ ]:
for roots, directories, documents in os.walk(output_dir):
    dir_path = os.path.basename(roots)
    #source_path = roots
    for document in documents:
        document_zip = document
        
        print(f'{roots}   {dir_path} {document_zip} \n')
        #logging.info("Processando arquivo: %s", document_path_1

In [ ]:
import os
from outlook_msg import Message

# Caminho da pasta onde você quer salvar os anexos
pasta_destino = msg_attachment_zip

# Verifica se a pasta existe; se não, cria ela
if not os.path.exists(pasta_destino):
    os.makedirs(pasta_destino)

with open(email_path) as msg_file:
    msg = Message(msg_file)
